In [6]:
from enum import Enum

#define a board with size N * N
N = 15

class BoardState(Enum):
	#three possible states for a given intersection
    EMPTY = 0
    BLACK = 1
    WHITE = 2

In [7]:
#Gomoku board design based on xerwin's tutorial
#http://www.cnblogs.com/erwin/p/7828956.html

class Gomoku(object):
    def __init__(self):
        #create a N * N map
        self.__chessMap = [[BoardState.EMPTY for j in range(N)] for i in range(N)]
        self.__currentI = -1
        self.__currentJ = -1
        self.__currentState = BoardState.EMPTY

    def get_chessMap(self):
        return self.__chessMap

    def get_chessboard_state(self, i, j):
        return self.__chessMap[i][j]

    def set_chessboard_state(self, i, j, state):
        self.__chessMap[i][j] = state
        self.__currentI = i
        self.__currentJ = j
        self.__currentState = state

    def get_chess_result(self):
        if self.connected_five(self.__currentI, self.__currentJ, self.__currentState):
            return self.__currentState
        else:
            return BoardState.EMPTY

    def direction_count(self, i, j, xdirection, ydirection, player):
        count = 0
        for step in range(1, 5): #look four more steps on a certain direction
            if xdirection != 0 and (j + xdirection * step < 0 or j + xdirection * step >= N):
                break
            if ydirection != 0 and (i + ydirection * step < 0 or i + ydirection * step >= N):
                break
            if self.__chessMap[i + ydirection * step][j + xdirection * step] == player:
                count += 1
            else:
                break
        return count

    def connected_five(self, i, j, player):
        #four directions: horizontal, vertical, two diagonals
        directions = [[(-1, 0), (1, 0)], \
                      [(0, -1), (0, 1)], \
                      [(-1, 1), (1, -1)], \
                      [(-1, -1), (1, 1)]]

        for axis in directions:
            axis_count = 1
            for (xdirection, ydirection) in axis:
                axis_count += self.direction_count(i, j, xdirection, ydirection, player)
                if axis_count >= 5:
                    return True

        return False


In [8]:
#based on xerwin's tutorial and pygame documentation

import pygame
from pygame.locals import *


IMAGE_PATH = 'UI/'

WIDTH = 540
HEIGHT = 540
MARGIN = 22
GRID = (WIDTH - 2 * MARGIN) / (N - 1)
PIECE = 32

class GameRender(object):
    def __init__(self, gomoku):
        self.__gomoku = gomoku

        #black starts first
        self.__currentPieceState = BoardState.BLACK

        #initialize pygame
        pygame.init()

        self.__screen = pygame.display.set_mode((WIDTH, HEIGHT), 0, 32)
        pygame.display.set_caption('Gomoku AI')

        #load UI resources 
        self.__ui_chessboard = pygame.image.load(IMAGE_PATH + 'chessboard.jpg').convert()
        self.__ui_piece_black = pygame.image.load(IMAGE_PATH + 'piece_black.png').convert_alpha()
        self.__ui_piece_white = pygame.image.load(IMAGE_PATH + 'piece_white.png').convert_alpha()

    def coordinate_transform_map2pixel(self, i, j):    
        #transform chessMap coordinates to UI
        return MARGIN + j * GRID - PIECE / 2, MARGIN + i * GRID - PIECE / 2

    def coordinate_transform_pixel2map(self, x, y):    
        #transform UI coordinates to chessMap
        i , j = int(round((y - MARGIN + PIECE / 2) / GRID)), int(round((x - MARGIN + PIECE / 2) / GRID))

        if i < 0 or i >= N or j < 0 or j >= N:
            return None, None
        else:
            return i, j

    def draw_chess(self):
        #board
        self.__screen.blit(self.__ui_chessboard, (0,0))
        #chess piece
        for i in range(0, N):
            for j in range(0, N):
                x,y = self.coordinate_transform_map2pixel(i,j)
                state = self.__gomoku.get_chessboard_state(i,j)
                if state == BoardState.BLACK:
                    self.__screen.blit(self.__ui_piece_black, (x,y))
                elif state == BoardState.WHITE:
                    self.__screen.blit(self.__ui_piece_white, (x,y))
                else: # BoardState.EMPTY
                    pass
                
    def draw_mouse(self):
        #track the mouse pointer
        x, y = pygame.mouse.get_pos()
        #chess piece moves with the mouse
        if self.__currentPieceState == BoardState.BLACK:
            self.__screen.blit(self.__ui_piece_black, (x - PIECE / 2, y - PIECE / 2))
        else:
            self.__screen.blit(self.__ui_piece_white, (x - PIECE / 2, y - PIECE / 2))

    def draw_result(self, result):
        font = pygame.font.SysFont('Arial', 55)
        tips = "Game Over:"
        if result == BoardState.BLACK :
            tips = tips + "Black Wins"
        elif result == BoardState.WHITE:
            tips = tips + "White Wins"
        else:
            tips = tips + "Draw"
        text = font.render(tips, True, (0, 0, 255))
        self.__screen.blit(text, (WIDTH / 2 - 200, HEIGHT / 2 - 50))

    def one_step(self):
        i, j = None, None
        #mouse click
        mouse_button = pygame.mouse.get_pressed()
        #left click
        if mouse_button[0]:
            x, y = pygame.mouse.get_pos()
            i, j = self.coordinate_transform_pixel2map(x, y)

        if not i is None and not j is None:
            #overlapped piece
            if self.__gomoku.get_chessboard_state(i, j) != BoardState.EMPTY:
                return False
            else:
                self.__gomoku.set_chessboard_state(i, j, self.__currentPieceState)
                return True

        return False
            
    def change_state(self):
        if self.__currentPieceState == BoardState.BLACK:
            self.__currentPieceState = BoardState.WHITE
        else:
            self.__currentPieceState = BoardState.BLACK

In [9]:
a = BoardState.WHITE
a.name



'WHITE'

In [10]:
gomoku = Gomoku()
vectors = []


for i in xrange(N):
    vectors.append(gomoku.get_chessMap()[i])

print(len(vectors))

for j in xrange(N):
    vectors.append([gomoku.get_chessMap()[i][j] for i in range(N)])

print(len(vectors))


vectors.append([gomoku.get_chessMap()[x][x] for x in range(N)])

for i in xrange(1, N-4):
    v = [gomoku.get_chessMap()[x][x-i] for x in range(i, N)]
    vectors.append(v)
    v = [gomoku.get_chessMap()[y-i][y] for y in range(i, N)]
    vectors.append(v)

print(len(vectors))

vectors.append([gomoku.get_chessMap()[x][N-x-1] for x in range(N)])

for i in xrange(4, N-1):
    v = [gomoku.get_chessMap()[x][i-x] for x in xrange(i, -1, -1)]
    vectors.append(v)
    v = [gomoku.get_chessMap()[x][N-x+N-i-2] for x in xrange(N-i-1, N)]
    vectors.append(v)
    
vector_t = (vectors)[0]

if [BoardState.EMPTY, BoardState.EMPTY, BoardState.EMPTY] in vector_t:
    print 'Yes'



15
30
51


In [28]:
def enum_to_string(vector):
    string_list = []
    for item in vector:
        if item == BoardState.BLACK:
            string_list.append('black')
        elif item == BoardState.WHITE:
            string_list.append('white')
        else:
            string_list.append('empty')
    
    return string_list
    
a = enum_to_string(vector_t)
print(a)

['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']


In [31]:
WHITE_6PATTERNS = [['empty', 'white', 'white', 'white', 'white','empty'],
                   ['empty', 'white', 'white', 'white', 'empty','empty'],
                   ['empty', 'empty', 'white', 'white', 'white','empty'],
                   ['empty', 'white', 'white', 'empty', 'white','empty'],
                   ['empty', 'white', 'empty', 'white', 'white','empty'],
                   ['empty', 'empty', 'white', 'white', 'empty','empty'],
                   ['empty', 'empty', 'white', 'empty', 'white','empty'],
                   ['empty', 'white', 'empty', 'white', 'empty','empty'],
                   ['empty', 'empty', 'white', 'empty', 'empty','empty'],
                   ['empty', 'empty', 'empty', 'white', 'empty','empty']]

WHITE_6SCORES = [10000,800,800,800,800,150,150,150,30,30]

WHITE_5PATTERNS = [['white', 'white', 'white', 'white', 'white'],
                   ['white', 'white', 'white', 'white', 'empty'],
                   ['empty', 'white', 'white', 'white', 'white'],
                   ['white', 'white', 'empty', 'white', 'white'],
                   ['white', 'empty', 'white', 'white', 'white'],
                   ['white', 'white', 'white', 'empty', 'white']]
WHITE_5SCORES = [100000,800,800,800,800,800]

BLACK_6PATTERNS = [['empty', 'black', 'black', 'black', 'black','empty'],
                   ['empty', 'black', 'black', 'black', 'empty','empty'],
                   ['empty', 'empty', 'black', 'black', 'black','empty'],
                   ['empty', 'black', 'black', 'empty', 'black','empty'],
                   ['empty', 'black', 'empty', 'black', 'black','empty'],
                   ['empty', 'empty', 'black', 'black', 'empty','empty'],
                   ['empty', 'empty', 'black', 'empty', 'black','empty'],
                   ['empty', 'black', 'empty', 'black', 'empty','empty'],
                   ['empty', 'empty', 'black', 'empty', 'empty','empty'],
                   ['empty', 'empty', 'empty', 'black', 'empty','empty']]
BLACK_6SCORES = [10000,800,800,800,800,150,150,150,30,30]

BLACK_5PATTERNS = [['black', 'black', 'black', 'black', 'black'],
                   ['black', 'black', 'black', 'black', 'empty'],
                   ['empty', 'black', 'black', 'black', 'black'],
                   ['black', 'black', 'empty', 'black', 'black'],
                   ['black', 'empty', 'black', 'black', 'black'],
                   ['black', 'black', 'black', 'empty', 'black']]
BLACK_5SCORES = [100000,800,800,800,800,800]

def evaluate_vector(vector):
    string_list = enum_to_string(vector)
    score = {'white':0,'black':0}
    length = len(string_list)
    
    
    if length == 5:
        for i in range(len(WHITE_5PATTERNS)):
            if WHITE_5PATTERNS[i] == string_list:
                score['white'] += WHITE_5SCORES[i]
            if BLACK_5PATTERNS[i] == string_list:
                score['black'] += BLACK_5SCORES[i]
        return score

    for i in range(length-5):
        temp = [string_list[i],string_list[i+1],string_list[i+2],string_list[i+3],string_list[i+4]]
        for i in range(len(WHITE_5PATTERNS)):
            if WHITE_5PATTERNS[i] == temp:
                score['white'] += WHITE_5SCORES[i]
            if BLACK_5PATTERNS[i] == temp:
                score['black'] += BLACK_5SCORES[i]
                
    for i in range(length-6):
        temp = [string_list[i],string_list[i+1],string_list[i+2],string_list[i+3],string_list[i+4],string_list[i+5]]
        for i in range(len(WHITE_6PATTERNS)):
            if WHITE_6PATTERNS[i] == temp:
                score['white'] += WHITE_6SCORES[i]
            if BLACK_6PATTERNS[i] == temp:
                score['black'] += BLACK_6SCORES[i]
                
    return score
        

In [33]:
evaluate_vector([BoardState.BLACK,BoardState.WHITE,BoardState.WHITE,BoardState.EMPTY,BoardState.WHITE,BoardState.BLACK,BoardState.WHITE])

{'black': 0, 'white': 0}

In [ ]:
class Node(object):
	def __init__(self, gomoku, depth, boardState):
		self.__gomoku = gomoku
		self.__currentState = boardState
		self.__depth = depth
		self.__currentI = -1
		self.__currentJ = -1

	def evaluate(self):
		vectors = []
        
        for i in xrange(N):
            vectors.append(self.__gomoku.get_chessMap()[i])
            
        for j in xrange(N):
            vectors.append([self.__gomoku.get_chessMap()[i][j] for i in range(N)])

        vectors.append([self.table.table[x][x] for x in range(0,row)])
        for i in xrange(1, row-4):
            vec = [self.table.table[x][x-i] for x in range(i, row)]
            vectors.append(vec)
            vec = [self.table.table[y-i][y] for y in range(i, col)]
            vectors.append(vec)
            #print [(y-i,y) for y in range(i, col)]

        # 1.4 '/'*21
        #vectors.append([self.table.table[x][row-x-1] for x in range(0, row)])
        #print [(x, row-x-1) for x in xrange(0, row)]
        for i in xrange(4, row-1):
            vec = [self.table.table[x][i-x] for x in xrange(i, -1, -1)]
            vectors.append(vec)
            vec = [self.table.table[x][col-x+row-i-2] for x in xrange(row-i-1, row)]
            vectors.append(vec)
            #print [(x,i-x) for x in xrange(i,-1,-1)]
            #print [(x,col-x+row-i-2) for x in xrange(row-i-1, row)]

In [5]:
import random


class gomokuAI(object):
	def __init__(self, gomoku, currentState):
		self.__gomoku = gomoku
		self.__currentState = currentState

	def one_step(self):
		i = random.randrange(0,15,1)
		j = random.randrange(0,15,1)

		if not i is None and not j is None:
			if self.__gomoku.get_chessboard_state(i, j) != BoardState.EMPTY:
				self.one_step()
			else:
				self.__gomoku.set_chessboard_state(i, j, self.__currentState)
				return True
		return False


In [10]:
from sys import exit

if __name__ == '__main__': 
    gomoku = Gomoku()
    render = GameRender(gomoku)
    #save for adding AI later
    ai = gomokuAI(gomoku, BoardState.WHITE)
    result = BoardState.EMPTY
    enable_ai = True

    while True:
        #pygame event
        for event in pygame.event.get():
            #exit
            if event.type == QUIT:
                exit()
            elif event.type ==  MOUSEBUTTONDOWN:
                #play a step
                if render.one_step():
                    result = gomoku.get_chess_result()
                else:
                    continue
                if result != BoardState.EMPTY:
                    break
                if enable_ai:
                    ai.one_step()
                    result = gomoku.get_chess_result()
                else:
                    render.change_state()
        
        #draw
        render.draw_chess()
        render.draw_mouse()

        if result != BoardState.EMPTY:
            render.draw_result(result)

        #update
        pygame.display.update()

SystemExit: 

/Users/ZitongMao/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


ues
